In [1]:
import os 

os.getcwd()
os.chdir('../')
os.getcwd()

'/home/gfspet/ml-projects/kidney-disease'

In [ ]:
import sys
sys.path.append('/home/gfspet/ml-projects/kidney-disease/src')
from dataclasses import dataclass
from pathlib import Path 
from cnn_classifier.constants import * 
from cnn_classifier.utils.common import read_yaml, create_directories


# AUGMENTATION: True
# IMAGE_SIZE: [224, 224, 3]
# BATCH_SIZE: 16
# INCLUDE_TOP: False
# EPOCHS: 10
# CLASSES: 3
# WEIGHTS: imagenet
# LEARNING_RATE: 0.001
# CONV_LAYERS: [
#             {'filters': choice(64, 128), 
#             'kernel_size': (3, 3), 
#             'activation': 'relu', 
#             'padding': 'same', 
#             'use_maxpool': True
#             },
#             {'filters': choice(128, 256), 
#             'kernel_size': (3, 3), 
#             'activation': 'relu', 
#             'padding': 'same', 
#             'use_maxpool': False
#             }
#         ]
# DENSE_LAYERS: [512, 256]
# DROPOUT: [0.1, 0.2, 0.3]

@dataclass(frozen=True)
class PretrainedModelConfig: 
    root_dir: Path
    base_model_path: Path
    used_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_conv_layers: list
    params_dense_layers: list
    params_dropout_rates: list
    params_classes: int
    
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_pretrained_base_model_config(self) -> PretrainedModelConfig:
        config = self.config.pretrained_base_model
        
        pretrained_base_model_config = PretrainedModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            used_model_path=Path(config.used_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP, 
            params_learning_rate=self.params.LEARNING_RATE,
            params_weights=self.params.WEIGHTS,
            params_conv_layers=self.params.CONV_LAYERS,
            params_dense_layers=self.params.DENSE_LAYERS,
            params_dropout_rates=self.params.DROPOUT,
            params_classes=self.params.CLASSES
        )
        
        return pretrained_base_model_config

In [ ]:
import tensorflow as tf

class PretrainedBaseModel: 
    def __init__(self, config: PretrainedModelConfig):
        self.config = config
    
    def get_base_model(self): 
        self.model = tf.keras.applications.MobileNetV2(
            input_shape=self.config.params_image_size, 
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            base_model=self.model,
            classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate
        )
        
        self.save_model(path=self.config.used_model_path, model=self.full_model)
        
    @staticmethod
    def prepare_full_model(base_model, classes, learning_rate): 
        for layer in base_model.layers:
            layer.trainable = False
            
        conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(base_model.output)
        maxpool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
        conv2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(maxpool1)
        maxpool2 = tf.keras.layers.MaxPooling2D((2,2))(conv2)
        avg_pool = tf.keras.layers.GlobalAveragePooling2D()(maxpool2)
        dense1 = tf.keras.layers.Dense(512, activation='relu')(avg_pool)
        batch_norm = tf.keras.layers.BatchNormalization()(dense1)
        dropout = tf.keras.layers.Dropout(0.2)(batch_norm)
        flatten = tf.keras.layers.Flatten()(dropout)
        dense2 = tf.keras.layers.Dense(256, activation='relu')(flatten)
        fc = tf.keras.layers.Dense(units=classes, activation='softmax')(dense2)
            
        full_model = tf.keras.models.Model(
            inputs=base_model.input,
            outputs=fc
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )
        
        # full_model.summary()
        return full_model
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model): 
        model.save(path)

2024-11-24 00:21:00.743486: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 00:21:00.863846: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 00:21:00.993680: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732378861.052465  380500 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732378861.084231  380500 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 00:21:01.316858: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [4]:
try:
    config = ConfigurationManager() 
    pretrained_base_model_config = config.get_pretrained_base_model_config()
    pretrained_base_model = PretrainedBaseModel(config=pretrained_base_model_config) 
    pretrained_base_model.get_base_model()
    pretrained_base_model.update_base_model()
except Exception as e: 
    raise e

[ 2024-11-24 00:21:05,751 ] 21 cnn_classifier_logger : INFO : common : yaml file: config/config.yaml loaded successfully
[ 2024-11-24 00:21:05,754 ] 21 cnn_classifier_logger : INFO : common : yaml file: params.yaml loaded successfully
[ 2024-11-24 00:21:05,756 ] 33 cnn_classifier_logger : INFO : common : Created directory at: artifacts


2024-11-24 00:21:05.764705: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


[ 2024-11-24 00:21:06,733 ] 83 absl : WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,269,379 (12.47 MB)

 Trainable params: 1,010,371 (3.85 MB)

 Non-trainable params: 2,259,008 (8.62 MB)

[ 2024-11-24 00:21:07,166 ] 83 absl : WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


In [ ]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from pathlib import Path
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Paths and parameters
base_dir = '/home/gfspet/ml-projects/kidney-disease/artifacts/data_ingestion/data'
subdirs = ['Stone', 'Tumor', 'Normal']
img_size = (224, 224)
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load and preprocess data
def load_data(base_dir, subdirs, img_size):
    images, labels = [], []
    for idx, subdir in enumerate(subdirs):
        subdir_path = Path(base_dir) / subdir
        for img_file in os.listdir(subdir_path):
            img_path = subdir_path / img_file
            img = load_img(img_path, target_size=img_size)  # Resize to img_size
            img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
            images.append(img_array)
            labels.append(idx)
    return np.array(images), np.array(labels)

# Load the data
images, labels = load_data(base_dir, subdirs, img_size)
labels = tf.keras.utils.to_categorical(labels, num_classes=len(subdirs))  # One-hot encoding

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data generators
train_datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=10)
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

# Load base model
def get_base_model(): 
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=[224, 224, 3], 
        # weights='/home/gfspet/ml-projects/kidney-disease/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5',
        include_top=False,
        weights='imagenet'
    )
    return base_model

# Prepare full model
def prepare_full_model(base_model, classes, learning_rate): 
    for layer in base_model.layers:
        layer.trainable = False
    
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(base_model.output)
    maxpool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
    conv2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(maxpool1)
    maxpool2 = tf.keras.layers.MaxPooling2D((2,2))(conv2)
    avg_pool = tf.keras.layers.GlobalAveragePooling2D()(maxpool2)
    dense1 = tf.keras.layers.Dense(512, activation='relu')(avg_pool)
    batch_norm = tf.keras.layers.BatchNormalization()(dense1)
    dropout = tf.keras.layers.Dropout(0.2)(batch_norm)
    flatten = tf.keras.layers.Flatten()(dropout)
    dense2 = tf.keras.layers.Dense(256, activation='relu')(flatten)
    fc = tf.keras.layers.Dense(units=classes, activation='softmax')(dense2)
    
    full_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=fc
    )
    
    full_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return full_model

# Initialize model
base_model = get_base_model()
model = prepare_full_model(base_model, classes=len(subdirs), learning_rate=learning_rate)

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
    batch_size=batch_size
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")